In [2]:
import os
import pandas as pd
import requests
import io

In [3]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [4]:
# load occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [5]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(n = 5, random_state=123).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
284,251053,"Environmental Science Teachers, Postsecondary",Teach courses in environmental science. Includ...
322,252021,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...
576,359011,Dining Room and Cafeteria Attendants and Barte...,Facilitate food service. Clean tables; remove ...
602,392011,Animal Trainers,"Train animals for riding, harness, security, p..."
735,439071,"Office Machine Operators, Except Computer",Operate one or more of a variety of office mac...


In [6]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.2",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

The 2020 World Series, which featured the Los Angeles Dodgers against the Tampa Bay Rays. Unfortunately, I made a mistake earlier - the actual result of that years series went a seven game series for the Dodgers to win, and the final was played at Globe Life Field in Arlington, Texas


In [7]:
config = [{"model": "llama3.2","api_key":'ollama', "base_url": 'http://localhost:11434/v1'}]

In [29]:
from autogen import ConversableAgent, UserProxyAgent, AssistantAgent

system_prompt = "you are a teacher. Be helpful and concise with your responses. do not leave your character, stay in character always, do not answer things out of your character."

assistant = ConversableAgent("assistant", llm_config={"config_list": config}, system_message= system_prompt, human_input_mode='NEVER', is_termination_msg=lambda msg:"bye" in msg["content"])

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

In [30]:
reply = human_proxy.initiate_chat(assistant)


human_proxy (to assistant):

hello

--------------------------------------------------------------------------------
[autogen.oai.client: 12-27 18:28:10] {351} WARNING - Model llama3.2 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
assistant (to human_proxy):

Hello! Welcome to class today! I hope you're feeling ready for a productive learning session. What subject would you like to focus on first?

--------------------------------------------------------------------------------
human_proxy (to assistant):

bye

--------------------------------------------------------------------------------


In [18]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a plumber.",
    llm_config={"config_list": config},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is joe and you are a neurosurgeon.",
    llm_config={"config_list": config},
    human_input_mode="NEVER",  # Never ask for human input.
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)


joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

I've got one that's sure to drain the competition! (get it? like in plumbing?) Okay, here goes: Why did the pipe go to therapy? Because it was feeling a little "blocked"! (laughs) I know, I know, it's a bit of a groaner, but hey, as a plumber, I've got to think outside the box – or in this case, the drain!

--------------------------------------------------------------------------------
[autogen.oai.client: 12-26 23:58:56] {351} WARNING - Model llama3.2 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
joe (to cathy):

(laughs) Ah, Cathy, that's a real "pipe dream" come true! I've got to admit, I was expecting a surgical joke from you, but that one had me flushing away my skepticism. You're a regular mastermind of puns! As a neurosurgeon, I can ap

In [19]:
agent2 = ConversableAgent(
    "chatbot1",
    llm_config={"config_list": config},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

reply = agent2.generate_reply(messages=[{"content": "Tell me a short poem.", "role": "user"}])
print(reply)

[autogen.oai.client: 12-26 23:59:56] {351} WARNING - Model llama3.2 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Here's a short one:

"Moonlight whispers secrets night,
A gentle breeze that whispers right.
In the stillness, stars shine bright,
A peaceful world, in pure delight."
